<a href="https://colab.research.google.com/github/LazoYoung/ai-sanbox/blob/main/perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 파라미터 초기화
퍼셉트론에 필요한 파라미터 변수를 무작위로 초기화하자
- 입력값: X
- 가중치: W
- 편향치: b
- 목표값: y

In [56]:
import numpy as np
np.random.seed(42)

X = np.array([0.5, 0.5, 0.5])  # 입력값 = [0.5, 0.5, 0.5]
W = np.random.rand(3)          # 가중치 3개 무작위 초기화
b = np.random.rand(1)          # 편향치 1개 무작위 초기화
y = np.array([-1])             # 목표값 = -1

In [57]:
print("퍼셉트론 파라미터 초기값...")
print("W:", W)
print("b:", b)

퍼셉트론 파라미터 초기값...
W: [0.37454012 0.95071431 0.73199394]
b: [0.59865848]


## 예측 함수
입력값을 가중치와 곱하고 편향치와 더하여 출력값 z를 계산하고,
임계 함수에 통과시켜 예측값 y^을 도출하는 함수를 구현하자

In [58]:
# 예측 함수
def predict(X, W, b):
    z = np.dot(X, W) + b
    return z

# 임계 함수
def threshold(z):
    if z > 0:
        return np.array([1])
    else:
        return np.array([-1])

# 순전파 함수
def forward(X, W, b):
    z = predict(X, W, b)
    y_hat = threshold(z)
    return y_hat

In [59]:
z = predict(X, W, b)
y_hat = threshold(z)
print("출력값 z =", z)
print("예측값 y^ =", y_hat)
print("목표값 y =", y)

출력값 z = [1.62728267]
예측값 y^ = [1]
목표값 y = [-1]


## 학습 구현
모델의 가중치와 편향을 조절하여 예측 성능을 최적화하는 학습을 구현해보자

In [60]:
learning_rate = 0.1
num_epochs = 10

for epoch in range(num_epochs):
    y_hat = forward(X, W, b)
    error = y - y_hat
    W = W + learning_rate * error * X
    b = b + learning_rate * error
    print(f"[Epoch #{epoch + 1}] Error = {error}, W = {W}, b = {b}")

[Epoch #1] Error = [-2], W = [0.27454012 0.85071431 0.63199394], b = [0.39865848]
[Epoch #2] Error = [-2], W = [0.17454012 0.75071431 0.53199394], b = [0.19865848]
[Epoch #3] Error = [-2], W = [0.07454012 0.65071431 0.43199394], b = [-0.00134152]
[Epoch #4] Error = [-2], W = [-0.02545988  0.55071431  0.33199394], b = [-0.20134152]
[Epoch #5] Error = [-2], W = [-0.12545988  0.45071431  0.23199394], b = [-0.40134152]
[Epoch #6] Error = [0], W = [-0.12545988  0.45071431  0.23199394], b = [-0.40134152]
[Epoch #7] Error = [0], W = [-0.12545988  0.45071431  0.23199394], b = [-0.40134152]
[Epoch #8] Error = [0], W = [-0.12545988  0.45071431  0.23199394], b = [-0.40134152]
[Epoch #9] Error = [0], W = [-0.12545988  0.45071431  0.23199394], b = [-0.40134152]
[Epoch #10] Error = [0], W = [-0.12545988  0.45071431  0.23199394], b = [-0.40134152]


In [61]:
z = predict(X, W, b)
y_hat = forward(X, W, b)
print("Trained weights:", W)
print("Trained bias:", b)
print("Trained z:", z)
print("Predict y^:", y_hat)

Trained weights: [-0.12545988  0.45071431  0.23199394]
Trained bias: [-0.40134152]
Trained z: [-0.12271733]
Predict y^: [-1]
